In [26]:
import numpy as np
import pandas as pd

# Parameters
num_samples = 200
min_time = 10
max_time = 15
min_temp = 160
max_temp = 300

# Generate random times and temperatures
np.random.seed(0)  # For reproducibility
x1 = np.random.randint(min_time, max_time + 1, num_samples)
x2= np.random.randint(min_temp, max_temp + 1, num_samples)

# Combine x1 and x2 into a 2D array
X = np.column_stack((x1, x2))

# Generate binary target values
Y = np.random.randint(0, 2, num_samples).reshape(-1, 1)

In [27]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf
import tensorflow.keras.layers as layers


In [28]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min pre normalization: 15.00, 10.00
Duration    Max, Min pre normalization: 300.00, 160.00
Temperature Max, Min post normalization: 1.54, -1.39
Duration    Max, Min post normalization: 1.77, -1.64


In [29]:
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape)   
model = tf.keras.Sequential([
    tf.keras.Input(shape=(2,)),
    layers.Dense(3, activation='sigmoid', name = 'layer1'),
    layers.Dense(1, activation='sigmoid', name = 'layer2')
])

(200000, 2) (200000, 1)


In [30]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 3)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
L1_num_params = 2 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters  + b2 parameters
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

L1 params =  9 , L2 params =  4
W1(2, 3):
 [[ 0.52934    -0.6570983  -0.85779834]
 [-0.19965404  0.9421389   0.66064477]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.8552693 ]
 [ 1.155985  ]
 [-0.87800777]] 
b2(1,): [0.]


In [32]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)


Epoch 1/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 246us/step - loss: 0.6887
Epoch 2/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 292us/step - loss: 0.6789
Epoch 3/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 301us/step - loss: 0.6721
Epoch 4/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 248us/step - loss: 0.6672
Epoch 5/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 312us/step - loss: 0.6651
Epoch 6/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 272us/step - loss: 0.6627
Epoch 7/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 333us/step - loss: 0.6632
Epoch 8/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 1s 228us/step - loss: 0.6620
Epoch 9/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 277us/step - loss: 0.6611
Epoch 10/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 2s 251us/step - loss: 0.6606


In [33]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1 = np.array([
    [-8.94,  0.29, 12.89],
    [-0.17, -7.34, 10.79]] )
b1 = np.array([-9.87, -9.28,  1.01])
W2 = np.array([
    [-31.38],
    [-27.86],
    [-32.79]])
b2 = np.array([15.54])
model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])

W1:
 [[-1.0113816 -7.214147  -1.7784194]
 [ 1.5105366 44.161186  10.433374 ]] 
b1: [-4.545078   4.546585   1.3059771]
W2:
 [[-4.3763623]
 [ 6.202392 ]
 [-5.9260263]] 
b2: [0.07781837]


In [34]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
predictions = 
 [[2.5643404e-20]
 [2.5643404e-20]]


In [35]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[0]
 [0]]


In [37]:
import matplotlib.pyplot as plt
# Plotting
plt.figure(figsize=(10, 6))

# Plot training data
plt.scatter(Xt[Yt == 0][:, 0], Xt[Yt == 0][:, 1], color='blue', label='Class 0')
plt.scatter(Xy[Yt == 1][:, 0], Xt[Yt == 1][:, 1], color='red', label='Class 1')

# Plot test data
plt.scatter(X_testn[:, 0], X_testn[:, 1], color='black', marker='x', s=100, label='Test Data')

# Plot decision boundary
x_min, x_max = Xt[:, 0].min() - 1, Xt[:, 0].max() + 1
y_min, y_max = Xt[:, 1].min() - 1, Xt[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))
Z = model.predict(norm_l(np.c_[xx.ravel(), yy.ravel()]))
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
plt.colorbar()
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Logistic Regression Decision Boundary')
plt.legend()
plt.show()


IndexError: boolean index did not match indexed array along dimension 1; dimension is 2 but corresponding boolean dimension is 1

<Figure size 1000x600 with 0 Axes>